# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.visual import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon.lexicon_memoris import lexicon_memoris, lithology_memoris, contamination_benzene_memoris
from utils.io import boreholes_from_files, gdf_viewer, boreholes_from_dataframe, striplog_from_dataframe
from core.project import Project
from core.orm import BoreholeOrm, PositionOrm, Base, IntervalDataOrm
import random
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

In [2]:
def symbols_from_attributes(attributes, default_lexicon, default_legend):
    symbols_dict = {}
    for k, v in attributes.items():    
        if 'legend' not in v.keys(): 
            v['legend'] = Legend.from_csv(text=default_legend.replace('attribute', k))
        if 'lexicon' not in v.keys():
            lexicon = default_lexicon.copy()
            lexicon.update({k: lexicon['attribute']})
            lexicon.pop('attribute', None)
            v['lexicon'] = Lexicon(lexicon)
        symbols_dict.update({k.lower(): {'legend': v['legend'], 'lexicon': v['lexicon']}})
    return symbols_dict

In [3]:
pollutants=['Benzene']
# default contamination level lexicon for pollutants
contaminations_lexicon = {'attribute': ['VR', 'VS', 'VI', 'INCONNU'], 'usage': ['naturel', 'agricole', 'r[é|e]sidentiel', 'commercial','r[é|e]cr[é|e]atif' 'industriel', ]}

In [4]:
contaminations_legend = "colour,width,component attribute\n#00FF00, None, VR,\n#FFA500, None, VS,\n#FF0000, None, VI,\n#FFFFFF, None, Inconnu\n"

In [5]:
memoris_symbols = symbols_from_attributes({'lithology': {'lexicon' : lexicon_memoris, 'legend': lithology_memoris}, 'benzene': {}, 'Toluene': {}}, default_lexicon=contaminations_lexicon, default_legend=contaminations_legend)

## Reading of CSV files in a dir

In [6]:
#df = pd.read_csv('test_data_boreholes.csv', sep=',') # use english lexicon
df = pd.read_csv('test_litho_fr.csv', sep=',') # use french lexicon

## Data Checking

In [7]:
#pol = ['As', 'Ni', 'Cl', 'Hg', 'Pb', 'Zn']
pol = ['VR','VS','VI']
#df[['Pollutant', 'Concentration']] = ''
df[['Pollutant', 'Concentration']] = ''
df['Pollutant'] = df['Pollutant'].apply(lambda x: random.sample(pol, 1)[0])
df['Concentration'] = df['Concentration'].apply(lambda x: np.random.randint(0,100))
df['Unit'] = 'mg/l'

In [8]:
df['Lithology'] = df['Lithology'].replace({'Grès':'Marne', 'Calcaire':'Dolomie'})

In [9]:
df.loc[35.5, :] = ['BH007',152794.0, 121111.0, 60.0, 0.57, 45, 'Calcaire', 'Bleu', 4, 41, 45, 'Cl', 78, 'mg/l']
df.loc[25.5, :] = ['BH005',152473.0, 121460.0, 72.0, 0.61, 32, 'Schiste', 'gris', 3, 29, 32, 'Cl', 78, 'mg/l']
df.drop(index=4, inplace=True)
df.drop(columns='Colour', inplace=True)
df = df.sort_index().reset_index(drop=True)

In [10]:
gdf_viewer(df)

Rows : 996, columns : 13


interactive(children=(IntSlider(value=10, description='rows', max=996, min=10, readout=False), IntSlider(value…

In [11]:
df['Benzene'] = df.Pollutant

In [12]:
tmp = df.query('ID=="BH001"')

In [13]:
memoris_symbols

{'lithology': {'legend': Legend(Decor({'_colour': '#ffffe9', 'width': None, 'hatch': None, 'component': Component({'lithology': 'matériau(?:x)? meuble(?:s)?'})})
  Decor({'_colour': '#ffffd5', 'width': None, 'hatch': "'....'", 'component': Component({'lithology': 'alluvion(?:s)?'})})
  Decor({'_colour': '#331100', 'width': None, 'hatch': None, 'component': Component({'lithology': 'boue(?:s)?'})})
  Decor({'_colour': '#d3b798', 'width': None, 'hatch': "'v'", 'component': Component({'lithology': 'remblai(?:s)?'})})
  Decor({'_colour': '#ffcc99', 'width': None, 'hatch': None, 'component': Component({'lithology': 'tourbe(?:s)?'})})
  Decor({'_colour': '#ffeaa7', 'width': None, 'hatch': None, 'component': Component({'lithology': 'gypse(?:s)?'})})
  Decor({'_colour': '#00151a', 'width': None, 'hatch': None, 'component': Component({'lithology': 'houille(?:s)?'})})
  Decor({'_colour': '#798732', 'width': None, 'hatch': "'-.'", 'component': Component({'lithology': 'limon(?:s)?'})})
  Decor({'_c

In [14]:
strip = striplog_from_dataframe(tmp, bh_name='BH001', attributes=['Lithology', 'Benzene'], symbols=memoris_symbols)

|__ID:'BH001'
[Component({'lithology': 'sable'}), Component({'benzene': 'vi'})]
[Component({'lithology': 'argile'}), Component({'benzene': 'vs'})]
[Component({'lithology': 'marne'}), Component({'benzene': 'vr'})]
[Component({'lithology': 'charbon'}), Component({'benzene': 'vs'})]
Summary : [Striplog(4 Intervals, start=0.0, stop=28.0)]


In [15]:
strip['BH001'][0].components

[Component({'lithology': 'sable'}), Component({'benzene': 'vi'})]

In [16]:
def get_components(strip):
    components = []
    for v in strip.values():
        components.extend([comp for iv in v._Striplog__list for comp in iv.components])
    return list(set(components))

## Create Boreholes from dataframes or files

In [17]:
bhs_df=[df[0:50]] # list of dataframes

In [18]:
boreholes, components, link_intv_comp = boreholes_from_dataframe(df[0:50], attributes=['Lithology', 'Benzene'], symbols=memoris_symbols)


Dataframe processing...
|__ID:'BH001'
[Component({'lithology': 'sable'}), Component({'benzene': 'vi'})]
[Component({'lithology': 'argile'}), Component({'benzene': 'vs'})]
[Component({'lithology': 'marne'}), Component({'benzene': 'vr'})]
[Component({'lithology': 'charbon'}), Component({'benzene': 'vs'})]
Summary : [Striplog(4 Intervals, start=0.0, stop=156.0)]
comp: {'lithology': 'sable'}, id: 1
comp: {'lithology': 'argile'}, id: 2
comp: {'lithology': 'charbon'}, id: 3
comp: {'benzene': 'vi'}, id: 4
comp: {'benzene': 'vs'}, id: 5
comp: {'benzene': 'vr'}, id: 6
comp: {'lithology': 'marne'}, id: 7
comp_dict: {Component({'lithology': 'sable'}): 0, Component({'lithology': 'argile'}): 1, Component({'lithology': 'charbon'}): 2, Component({'benzene': 'vi'}): 3, Component({'benzene': 'vs'}): 4, Component({'benzene': 'vr'}): 5, Component({'lithology': 'marne'}): 6}
comp_dict: {Component({'lithology': 'sable'}): 0, Component({'lithology': 'argile'}): 1, Component({'lithology': 'charbon'}): 2, Co

|__ID:'BH007'
[Component({'lithology': 'sable'}), Component({'benzene': 'vi'})]
[Component({'lithology': 'argile'}), Component({'benzene': 'vi'})]
[Component({'lithology': 'marne'}), Component({'benzene': 'vs'})]
[Component({'lithology': 'charbon'}), Component({'benzene': 'vr'})]
[Component({'lithology': 'dolomie'}), Component({'benzene': 'vr'})]
Error : No value matching with 'Cl' in given lexicon
[Component({'lithology': 'calcaire'})]
Summary : [Striplog(6 Intervals, start=0.0, stop=250.0)]
comp: {'lithology': 'argile'}, id: 9
comp: {'lithology': 'marne'}, id: 9
comp: {'lithology': 'charbon'}, id: 9
comp: {'lithology': 'dolomie'}, id: 9
comp: {'lithology': 'calcaire'}, id: 10
comp: {'benzene': 'vs'}, id: 10
comp: {'benzene': 'vi'}, id: 10
comp: {'benzene': 'vr'}, id: 10
comp: {'lithology': 'sable'}, id: 10
comp_dict: {Component({'lithology': 'sable'}): 0, Component({'lithology': 'argile'}): 1, Component({'lithology': 'charbon'}): 2, Component({'benzene': 'vi'}): 3, Component({'benzen

/home/su530201/PycharmProjects/GSDMA/utils/io.py:493: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   The 'overlaps' parameter may be used to remove this warning.
  boreholes.append(BoreholeOrm(id=bh_name))
/home/su530201/PycharmProjects/GSDMA/utils/io.py:493: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.int_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies I

In [19]:
boreholes

[<core.orm.BoreholeOrm>(Name=BH001, Length=156.0, Diameter=0.43, Intervals=4),
 <core.orm.BoreholeOrm>(Name=BH002, Length=185.0, Diameter=0.6, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH003, Length=90.0, Diameter=0.41, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH004, Length=130.0, Diameter=0.61, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH005, Length=177.0, Diameter=0.61, Intervals=6),
 <core.orm.BoreholeOrm>(Name=BH006, Length=130.0, Diameter=0.31, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH007, Length=250.0, Diameter=0.57, Intervals=6),
 <core.orm.BoreholeOrm>(Name=BH008, Length=175.0, Diameter=0.39, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH009, Length=200.0, Diameter=0.5, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH10, Length=124.0, Diameter=0.31, Intervals=4)]

In [20]:
components

{0: Component({'lithology': 'sable'}),
 1: Component({'lithology': 'argile'}),
 2: Component({'lithology': 'charbon'}),
 3: Component({'benzene': 'vi'}),
 4: Component({'benzene': 'vs'}),
 5: Component({'benzene': 'vr'}),
 6: Component({'lithology': 'marne'}),
 7: Component({'lithology': 'dolomie'}),
 8: Component({'lithology': 'schiste'}),
 9: Component({'lithology': 'calcaire'})}

In [21]:
len(boreholes), len(components)

(10, 10)

## Create a project and save data in a database

In [22]:
remove('tmp_files/tfe_orm_db.db')

In [23]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [24]:
Base.metadata.create_all(engine)

2021-07-08 19:33:28,765 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-08 19:33:28,768 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-07-08 19:33:28,769 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-08 19:33:28,771 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-07-08 19:33:28,772 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-08 19:33:28,774 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Positions")
2021-07-08 19:33:28,775 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-08 19:33:28,776 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Positions")
2021-07-08 19:33:28,777 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-08 19:33:28,778 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("IntervalData")
2021-07-08 19:33:28,780 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-08 19:33:28,782 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("IntervalData")
2021-07-08 19:33:28,783 INFO sqlalchemy.engine.E

In [25]:
Session = sessionmaker(bind=engine)
session = Session()

In [26]:
p = Project(session, legend_dict={'lithology':{'legend': lithology_memoris}, 'benzene': {'legend': contamination_benzene_memoris}}, 
            lexicon=lexicon_memoris, repr_attribute='lithology')

p.add_components(components)

2021-07-08 19:33:28,865 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-08 19:33:28,875 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-07-08 19:33:28,877 INFO sqlalchemy.engine.Engine [generated in 0.00231s] ()
2021-07-08 19:33:28,888 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-07-08 19:33:28,889 INFO sqlalchemy.engine.Engine [generated in 0.00114s] ((0, 'Sable'), (1, 'Argile'), (2, 'Charbon'), (3, 'Vi'), (4, 'Vs'), (5, 'Vr'), (6, 'Marne'), (7, 'Dolomie'), (8, 'Schiste'), (9, 'Calcaire'))
2021-07-08 19:33:28,897 INFO sqlalchemy.engine.Engine COMMIT
2021-07-08 19:33:28,904 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-08 19:33:28,906 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FR

Add boreholes into the project

In [27]:
for bh in boreholes:
    p.add_borehole(bh)

2021-07-08 19:33:28,921 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-07-08 19:33:28,923 INFO sqlalchemy.engine.Engine [generated in 0.00223s] ('BH001', 156.0, 0.43)
2021-07-08 19:33:28,927 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-07-08 19:33:28,928 INFO sqlalchemy.engine.Engine [generated in 0.00159s] ((0, 101.0, 101.0, 101.0, 152600.0, 121039.0), (1, 62.0, 62.0, 62.0, 152600.0, 121039.0), (2, 62.0, 62.0, 62.0, 152600.0, 121039.0), (3, 23.0, 23.0, 23.0, 152600.0, 121039.0), (4, 23.0, 23.0, 23.0, 152600.0, 121039.0), (5, -16.0, -16.0, -16.0, 152600.0, 121039.0), (6, -16.0, -16.0, -16.0, 152600.0, 121039.0), (7, -55.0, -55.0, -55.0, 152600.0, 121039.0))
2021-07-08 19:33:28,933 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?)
2021-07-08 19:33:28,934 INFO sq

2021-07-08 19:33:29,112 INFO sqlalchemy.engine.Engine [cached since 0.1333s ago] (8,)
2021-07-08 19:33:29,119 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-07-08 19:33:29,122 INFO sqlalchemy.engine.Engine [cached since 0.135s ago] (9,)
2021-07-08 19:33:29,127 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-07-08 19:33:29,129 INFO sqlalchemy.engine.Engine [cached since 0.1495s ago] (10,)
2021-07-08 19:33:29,134 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id",

2021-07-08 19:33:29,274 INFO sqlalchemy.engine.Engine [cached since 0.2948s ago] (22,)
2021-07-08 19:33:29,276 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-07-08 19:33:29,277 INFO sqlalchemy.engine.Engine [cached since 0.2905s ago] (23,)
2021-07-08 19:33:29,281 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-07-08 19:33:29,283 INFO sqlalchemy.engine.Engine [cached since 0.304s ago] (24,)
2021-07-08 19:33:29,286 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id

2021-07-08 19:33:29,381 INFO sqlalchemy.engine.Engine [cached since 0.4023s ago] (36,)
2021-07-08 19:33:29,384 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-07-08 19:33:29,386 INFO sqlalchemy.engine.Engine [cached since 0.399s ago] (37,)
Borehole geometry created successfully !
2021-07-08 19:33:29,414 INFO sqlalchemy.engine.Engine UPDATE "Boreholes" SET length=? WHERE "Boreholes".id = ?
2021-07-08 19:33:29,415 INFO sqlalchemy.engine.Engine [cached since 0.3464s ago] (75.0, 'BH004')
2021-07-08 19:33:29,424 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-07-08 19:33:29,425 INFO sqlalchemy.engine.Engine [cached since 0.5042s ago] ('BH005', 177.0, 0.61)
2021-07-08 19:33:29,427 INFO sql

2021-07-08 19:33:29,548 INFO sqlalchemy.engine.Engine [cached since 0.4799s ago] (72.0, 'BH005')
2021-07-08 19:33:29,553 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-07-08 19:33:29,556 INFO sqlalchemy.engine.Engine [cached since 0.6353s ago] ('BH006', 130.0, 0.31)
2021-07-08 19:33:29,558 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-07-08 19:33:29,559 INFO sqlalchemy.engine.Engine [cached since 0.6322s ago] ((50, 101.0, 101.0, 101.0, 152375.0, 121002.0), (51, 75.0, 75.0, 75.0, 152375.0, 121002.0), (52, 75.0, 75.0, 75.0, 152375.0, 121002.0), (53, 49.0, 49.0, 49.0, 152375.0, 121002.0), (54, 49.0, 49.0, 49.0, 152375.0, 121002.0), (55, 23.0, 23.0, 23.0, 152375.0, 121002.0), (56, 23.0, 23.0, 23.0, 152375.0, 121002.0), (57, -3.0, -3.0, -3.0, 152375.0, 121002.0), (58, -3.0, -3.0, -3.0, 152375.0, 121002.0), (59, -29.0, -29.0, -29.0, 152375.0, 121002.0))
2021-07-08 19:33

2021-07-08 19:33:29,656 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-08 19:33:29,657 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-07-08 19:33:29,658 INFO sqlalchemy.engine.Engine [cached since 0.7835s ago] ()
2021-07-08 19:33:29,661 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id", "Intervals".data_id AS "Intervals_data_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-07-08 19:33:29,663 INFO sqlalchemy.engine.Engine [cached since 0.6981s ago] ('BH007',)
2021-07-08 19:33:29,668 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Posi

2021-07-08 19:33:29,792 INFO sqlalchemy.engine.Engine [cached since 0.8271s ago] ('BH008',)
2021-07-08 19:33:29,795 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-07-08 19:33:29,796 INFO sqlalchemy.engine.Engine [cached since 0.8165s ago] (72,)
2021-07-08 19:33:29,800 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-07-08 19:33:29,802 INFO sqlalchemy.engine.Engine [cached since 0.8151s ago] (73,)
2021-07-08 19:33:29,806 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positi

2021-07-08 19:33:29,906 INFO sqlalchemy.engine.Engine [cached since 0.9195s ago] (85,)
2021-07-08 19:33:29,909 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-07-08 19:33:29,910 INFO sqlalchemy.engine.Engine [cached since 0.9312s ago] (86,)
2021-07-08 19:33:29,911 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-07-08 19:33:29,914 INFO sqlalchemy.engine.Engine [cached since 0.9272s ago] (87,)
2021-07-08 19:33:29,917 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_i

Borehole geometry created successfully !


In [28]:
p.add_link_between_components_and_intervals(link_intv_comp)

2021-07-08 19:33:30,076 INFO sqlalchemy.engine.Engine UPDATE "Boreholes" SET length=? WHERE "Boreholes".id = ?
2021-07-08 19:33:30,078 INFO sqlalchemy.engine.Engine [cached since 1.01s ago] (69.0, 'BH10')
2021-07-08 19:33:30,086 INFO sqlalchemy.engine.Engine INSERT INTO "Linkintervalcomponent" (int_id, comp_id, extra_data) VALUES (?, ?, ?)
2021-07-08 19:33:30,093 INFO sqlalchemy.engine.Engine [generated in 0.00787s] ((0, 0, ''), (0, 3, ''), (1, 1, ''), (1, 4, ''), (2, 6, ''), (2, 5, ''), (3, 2, ''), (3, 4, '')  ... displaying 10 of 98 total bound parameter sets ...  (49, 2, ''), (49, 5, ''))
2021-07-08 19:33:30,103 INFO sqlalchemy.engine.Engine COMMIT
2021-07-08 19:33:30,108 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-08 19:33:30,110 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-07-08 19:33:30,111 INFO sqlalchemy.engine.Engine [cached since 1

In [29]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

2021-07-08 19:33:30,123 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  10


In [30]:
p.boreholes

2021-07-08 19:33:30,147 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-08 19:33:30,154 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-07-08 19:33:30,158 INFO sqlalchemy.engine.Engine [generated in 0.00448s] ('BH001',)
2021-07-08 19:33:30,161 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id", "Intervals".data_id AS "Intervals_data_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-07-08 19:33:30,162 INFO sqlalchemy.engine.Engine [cached since 1.198s ago] ('BH001',)
2021-07-08 19:33:30,165 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_i

2021-07-08 19:33:30,236 INFO sqlalchemy.engine.Engine [cached since 1.271s ago] ('BH10',)


[<core.orm.BoreholeOrm>(Name=BH001, Length=62.0, Diameter=0.43, Intervals=4),
 <core.orm.BoreholeOrm>(Name=BH002, Length=66.0, Diameter=0.6, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH003, Length=85.0, Diameter=0.41, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH004, Length=75.0, Diameter=0.61, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH005, Length=72.0, Diameter=0.61, Intervals=6),
 <core.orm.BoreholeOrm>(Name=BH006, Length=75.0, Diameter=0.31, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH007, Length=60.0, Diameter=0.57, Intervals=6),
 <core.orm.BoreholeOrm>(Name=BH008, Length=66.0, Diameter=0.39, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH009, Length=60.0, Diameter=0.5, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH10, Length=69.0, Diameter=0.31, Intervals=4)]

In [31]:
bh = p.boreholes[0]

In [32]:
intv = bh.intervals[0]

In [33]:
data = {'key':'Benzene', 'value': 1.52, 'units': 'µg/l'}

In [34]:
benzene_1 = IntervalDataOrm(id=1, key='Benzene', value=1.52, units='µg/l')

In [35]:
benzene_1 = IntervalDataOrm(id=1, **data)

In [36]:
intv.data = benzene_1

In [37]:
p.refresh(verbose=False)

2021-07-08 19:33:30,298 INFO sqlalchemy.engine.Engine INSERT INTO "IntervalData" (id, interval, "key", value, units) VALUES (?, ?, ?, ?, ?)
2021-07-08 19:33:30,300 INFO sqlalchemy.engine.Engine [generated in 0.00230s] (1, None, 'Benzene', 1.52, 'µg/l')
2021-07-08 19:33:30,305 INFO sqlalchemy.engine.Engine UPDATE "Intervals" SET data_id=? WHERE "Intervals".id = ?
2021-07-08 19:33:30,306 INFO sqlalchemy.engine.Engine [generated in 0.00138s] (1, 0)
2021-07-08 19:33:30,308 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-07-08 19:33:30,308 INFO sqlalchemy.engine.Engine [cached since 1.434s ago] ()


In [38]:
session.commit()

2021-07-08 19:33:30,317 INFO sqlalchemy.engine.Engine COMMIT


In [39]:
session.close()

# 2D/3D Display

In [40]:
p.legend_dict.keys() #['pollutant']

dict_keys(['lithology', 'benzene'])

In [41]:
p.update_legend_cmap()

({'lithology': {'legend': Legend(Decor({'_colour': '#ffffff', 'width': 3, 'hatch': None, 'component': Component({'lithology': 'Inconnu'})})),
   'cmap': <matplotlib.colors.ListedColormap at 0x7fc512a94970>,
   'values': ['Inconnu']}},
 {'BH001': {'lithology': {'legend': Legend(Decor({'_colour': '#ffffff', 'width': 3, 'hatch': None, 'component': Component({'lithology': 'Inconnu'})})),
    'cmap': <matplotlib.colors.ListedColormap at 0x7fc512a94a60>,
    'values': ['Inconnu']}},
  'BH002': {'lithology': {'legend': Legend(Decor({'_colour': '#ffffff', 'width': 3, 'hatch': None, 'component': Component({'lithology': 'Inconnu'})})),
    'cmap': <matplotlib.colors.ListedColormap at 0x7fc512a93880>,
    'values': ['Inconnu']}},
  'BH003': {'lithology': {'legend': Legend(Decor({'_colour': '#ffffff', 'width': 3, 'hatch': None, 'component': Component({'lithology': 'Inconnu'})})),
    'cmap': <matplotlib.colors.ListedColormap at 0x7fc512a94c10>,
    'values': ['Inconnu']}},
  'BH004': {'lithology':

## Display one object (attribute = 'lithology')

In [43]:
bh = 1

In [46]:
p.boreholes_3d[bh].log_plot('lithology', repr_legend=memoris_symbols['lithology']['legend'])

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
p.boreholes_3d[bh].log_plot

In [ ]:
bh=1
p.boreholes_3d[bh].plot3d(show_legend=False)#, scalar_bar_args=sargs)

## Plot with another attribute

In [ ]:
p.update_legend_cmap(repr_attribute='pollutant')[1] #legend=lithology_memoris, update_legend=False)[0]

In [ ]:
p.boreholes_3d[bh].log_plot(repr_attribute='pollutant', repr_legend=pollutant_memoris)

In [ ]:
p.boreholes_3d[bh].plot3d(repr_attribute='pollutant', 
                          repr_legend=pollutant_memoris, 
                          show_legend=True)

### Display project boreholes

### Boreholes in 3D using X3D for interaction within the notebook